<img src="https://github.com/christopherhuntley/BUAN5405-docs/blob/master/Slides/img/Dolan.png?raw=true" width="180px" align="right">

# Lesson 11: pandas DataFrames
_The Swiss Army Knife of Data Science_

# Learning Objectives

## Theory / Be able to explain ...
- aaa

## Skills / Know how to  ...
- aaa

**This is new content you won't find in the Py4E book. Consider it a preview of your next course in Python for analytics. All roads eventually lead to pandas DataFrames.**